# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [7]:
w = data[data.race=='w']['call']
b = data[data.race=='b']['call']

In [8]:
# number of callbacks for white-sounding names
white_callback = sum(data[data.race=='w'].call)

In [9]:
# number of callbacks for black-sounding names
black_callback = sum(data[data.race=='b'].call)

In [10]:
len(w)

2435

In [11]:
len(b)

2435

# Question 1
Yes, the central limit theorem theory does apply. I would do either a permuatation test or a two proportion z test.

# Question 2
The null hypothesis is that the proportion of callbacks is the same for applications with black sounding and applications with white sounding names. The alternative hypothesis is that the proportion of callbacks between the two groups does differ.

In [12]:
# Your solution to Q3 here

# Question 3 
### Bootstrap approach below

In [13]:
difference = white_callback - black_callback

In [14]:
difference

78.0

In [15]:
np.random.seed(8)
differences = np.empty(10000)
for i in range(10000):
    all_perm = np.random.permutation(pd.concat([w, b]))
    whites = all_perm[:len(w)]
    blacks = all_perm[len(w):]
    differences[i] = sum(whites) - sum(blacks)


In [16]:
differences.max()

70.0

In [17]:
p = len(differences[differences > difference])/10000

In [18]:
p

0.0

### Frequentist approach below

In [21]:
from statsmodels.stats.proportion import proportions_ztest

In [22]:
z, p = proportions_ztest([white_callback, black_callback], [len(w),len(b)])

In [36]:
z

4.108412152434346

In [23]:
p

3.983886837585077e-05

In [43]:
standard_error = np.sqrt((((white_callback/len(w))*(1-(white_callback/len(w))))/len(w))+(((black_callback/len(b))*(1-(black_callback/len(b))))/len(b)))

In [40]:
black_callback/len(b)

0.06447638603696099

In [51]:
difference

78.0

In [55]:
lower_95 = white_callback/len(w)-black_callback/len(b) - standard_error * stats.norm.ppf(0.975)

In [56]:
upper_95 = white_callback/len(w)-black_callback/len(b) + standard_error * stats.norm.ppf(0.975)

In [57]:
print(lower_95,  upper_95)

0.016777728181230755 0.04728798023766041


In [58]:
margin_of_error = (upper_95 - lower_95)/2

In [59]:
margin_of_error

0.015255126028214829

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [52]:
white_callback/len(w)

0.09650924024640657

In [53]:
black_callback/len(b)

0.06447638603696099

In [54]:
white_callback/len(w)-black_callback/len(b)

0.032032854209445585

# Question 4

About 9.6 percent of the applicants with white sounding names in our sample got a callback, while the same is true for only about 6.4 percent of black applicants. The p value we got by the permutation approach as well as by the frequentist approach means that it is highly unlikely this difference is due to chance. The difference in callback rate is about 3.2 percent in favour of white-sounding names. There is a 95% chance that if those randomly assigned resumes would have been sent to all employers in the US labor market, the difference would have been between about 1.7 4.7 percent.

# Question 5
No, of course it doesn't say that, as its the only factor that was analyzed by the study. In order to know if its the most important factor, all possible factors would have to be tested. Of course that's not possible, but more factors could be tested. If the goal of the study was only to find out if this factor is the most influencial on callback (this wouldn't be a reasonable question to ask in my opinion though), I would choose a factor I expect to be as influencial as possible and repeat the experiment. Then I would do the experiment iteratively with more factors.